In [10]:
import pandas as pd

df = pd.read_csv(
    "/home/samuel/Desktop/Appunti/Insite/TensorFlow/opsd-household_data-2020-04-15/household_data_15min_singleindex.csv"
)


In [11]:
import pandas as pd
timestamp_column = "cet_cest_timestamp"

data_column = "DE_KN_industrial1_grid_import"

df = df[[timestamp_column, data_column]]

# Calcolo della differenza tra il valore corrente e quello precedente nella colonna DE_KN_industrial1_grid_import, DE_KN_industrial1_grid_import
df[f"{data_column}_DIFF"] = df[data_column].diff()

# Rimuove righe con campi vuoti
df.dropna(inplace=True)

df

,cet_cest_timestamp,DE_KN_industrial1_grid_import,DE_KN_industrial1_grid_import_DIFF
33745,2015-11-28T07:00:00+0100,4.375,3.875
33746,2015-11-28T07:15:00+0100,11.625,7.250
33747,2015-11-28T07:30:00+0100,18.750,7.125
33748,2015-11-28T07:45:00+0100,23.375,4.625
33749,2015-11-28T08:00:00+0100,26.875,3.500
...,...,...,...
99469,2017-10-12T23:00:00+0200,511090.617,8.375
99470,2017-10-12T23:15:00+0200,511097.586,6.969
99471,2017-10-12T23:30:00+0200,511104.305,6.719
99472,2017-10-12T23:45:00+0200,511112.023,7.718


In [16]:
from datetime import datetime, date, timedelta
import pandas as pd
import holidays

quarti_d_ora_in_una_giornata = 96


def get_week_number(date):
    iso_calendar = date.isocalendar()
    return iso_calendar[1]


"""
    1-> lun/ven, 2-> sabato, 3 -> domenica e festivi
"""
def determine_membership_category(dateToVerify, country="Germany"):
    """
    Determina la categoria di una data fornita in base al fatto che cada in un giorno festivo, una domenica o un normale giorno feriale.

    Args:
    dateToVerify (datetime.date): La data da verificare.

    Returns:
    str: Una stringa che rappresenta la categoria della data. Ritorna "3" per le festività o le domeniche, "1" per i giorni feriali, e "2" per altre tipologie di giorni non specificate.
    """
    weekday = dateToVerify.weekday()
    holiday_calendar = holidays.CountryHoliday(country)
    date_to_verify = date(dateToVerify.year, dateToVerify.month, dateToVerify.day)
    if (date_to_verify in holiday_calendar) or (weekday == 6):
        return 3  # Rappresenta le festività o le domeniche
    elif weekday >= 0 and weekday <= 4:
        return 1  # Rappresenta i giorni feriali
    return 2  # Categoria predefinita, potrebbe essere utilizzata per altri tipi di giorni


def calculate_quarter_hour(data: datetime, mode: str) -> int:
    """
    Calcola il quarto d'ora in cui cade una data specificata, in base alla modalità scelta.
    Le modalità possono essere "giorno", "settimana" o "mese", ma il conteggio è in quarti d'ora.

    Args:
    data (datetime): La data per la quale calcolare il quarto d'ora.
    modalità (str): La modalità di conteggio ("giorno", "settimana", "mese").

    Returns:
    int: Il quarto d'ora in cui cade la data, secondo la modalità scelta.
    """
    if mode == "day":
        minutes_since_start_of_day = data.hour * 60 + data.minute
        return minutes_since_start_of_day // 15 + 1
    elif mode == "week":
        minutes_since_start_of_week = (
            (data.weekday() * 24 * 60) + (data.hour * 60) + data.minute
        )
        return minutes_since_start_of_week // 15 + 1
    elif mode == "month":
        minutes_since_start_of_month = (
            ((data.day - 1) * 24 * 60) + (data.hour * 60) + data.minute
        )
        return minutes_since_start_of_month // 15 + 1
    else:
        return None

In [17]:
from dateutil import parser

for index, row in df.iterrows():
    try:
        datetime_obj = parser.parse(row[timestamp_column])
        week_number= get_week_number(datetime_obj)
        quarters= calculate_quarter_hour(datetime_obj, "week")
        quarters_dataset = calculate_quarter_hour(datetime_obj, "day")
        category= determine_membership_category(datetime_obj)
        df.at[index, "week_number"] = week_number
        df.at[index, "month_number"] = datetime_obj.month
        df.at[index, "quarters"] = quarters
        df.at[index, "category"] = category
        df.at[index, "fake_import_1"] = (4 - category) * (
            max(0, 1000 - (48 - quarters_dataset) ** 2)
        )
        df.at[index, "fake_import_2"] = (8 - datetime_obj.weekday()) * (
            max(0, 1000 - (48 - quarters_dataset) ** 2)
        )
        df.at[index, "timestamp"] = round(datetime_obj.timestamp())

    except Exception as e:
        print(e)

In [18]:
df.to_csv("debug.csv")

# Calcola l'indice per il 20% dei dati
split_idx = int(len(df) * 0.8)

# Suddividi il DataFrame in 80% train e 20% test senza mischiare
train = df[:split_idx]
test = df[split_idx:]

# Salva i dataset ottenuti
train.to_csv(
    "/home/antonio/Scrivania/Michele/Dataset energetico per test previsioni/V3/train.csv"
)
test.to_csv(
    "/home/antonio/Scrivania/Michele/Dataset energetico per test previsioni/V3/test.csv"
)

